In [24]:
import numpy as np
import plotly.io as pio
from processes.reserves import light_weight_reserve
from tqdm import tqdm

pio.renderers.default = 'iframe'

intensities = [{"warm_up_intensity": 1, "main_intensity": 10, "is_main": True},
               {"warm_up_intensity": 1, "main_intensity": 10, "is_main": False},
               {"warm_up_intensity": 1, "main_intensity": 10, "is_main": False},
               {"warm_up_intensity": 1, "main_intensity": 10, "is_main": False},
               {"warm_up_intensity": 1, "main_intensity": 10, "is_main": False}]

light_weight_reserve(intensities)

0.2917820218227087

$$\large{Task^1}$$\
$$\large{\lambda = (1, 1, 1, 1, 1)}$$\
$$\large{\Lambda = (10, 10, 10, 10, 10)}$$\
$$\large{main = 1}$$

In [25]:
def lln(experiments, n):
    return np.mean(experiments[:n + 1])


experiments = np.vectorize(lambda _: light_weight_reserve(intensities))(np.arange(1, 10001))
experiments_lln = np.vectorize(lambda n: lln(experiments, n))(np.arange(0, 10000))

$$\mathbb{E}\mathit{T_n} = \sum_{k=1}^{n}\frac{1}{\Lambda + (k-1)\lambda}$$

In [26]:
real_mean = np.sum(np.fromfunction(lambda i: 1 / (10 + i), (5,)))
print(real_mean)

0.42259407259407256


In [27]:
import plotly.express as px
N = np.arange(1, 10001)
fig = px.scatter(x=N, y=experiments_lln)

fig.update_layout(
    title=r"$\large{\text{Law of large numbers}}$",
    xaxis_title=r"$\Large{n}$",
    yaxis_title=r"$\Large{\frac{S_n}{n}}$",
)
fig.add_hline(y=real_mean, line_color="pink")


In [ ]:
fig = px.scatter(x=N[:1001], y=experiments_lln[:1001], animation_frame=N, range_y=[0, 0.8],range_x=[1, 10000])
fig.update_layout(
    title=r"$\large{\text{Law of large numbers}}$",
    xaxis_title=r"$\Large{n}$",
    yaxis_title=r"$\Large{\frac{S_n}{n}}$",
    yaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 0.05
    )
)
sliders = [dict(
    currentvalue=dict(prefix = r'n = ')
)]

fig.update_layout(
    sliders=sliders
)
fig.add_hline(y=real_mean, line_color="pink")


In [29]:
experiments_100 = [np.vectorize(lambda _: light_weight_reserve(intensities))(np.arange(1, 10001)) for i in tqdm(range(100))]
experiments_lln = [np.vectorize(lambda n: lln(experiments, n))(np.arange(0, 10000)) for experiments in experiments_100]


  3%|▎         | 3/100 [02:15<1:13:12, 45.29s/it]A

100%|██████████| 100/100 [15:39<00:00,  9.40s/it]


In [53]:
mean_errors_lln = np.mean(np.abs(experiments_lln - real_mean).T, axis=1)
mean_errors_lln

UFuncTypeError: ufunc 'subtract' did not contain a loop with signature matching types (dtype('<U16'), dtype('float64')) -> None

In [52]:
import plotly.graph_objects as go

fig = px.scatter(x=N[100:], y=mean_errors_lln[100:])

fig.update_layout(
    title=r"$\large{\text{Law of large numbers}}$",
    xaxis_title=r"$\Large{n}$",
    yaxis_title=r"$\Large{\frac{S_n}{n}}$",
)

fig.add_traces([go.Scatter(x=N[100:], y=0.15/np.sqrt(N[100:]))])

$$\mathbb{D}\mathit{T_n} = \mathbb{D}\sum_{k=1}^{n}\mathit{E}_{\Lambda + (k - 1)\lambda} = \sum_{k=1}^{n}\mathbb{D}\mathit{E}_{\Lambda + (k - 1)\lambda} = \sum_{k=1}^{n}\frac{1}{(\Lambda + (k - 1)\lambda)^2}$$

In [44]:
real_variance = np.sum(np.fromfunction(lambda i: 1 / (10 + i) ** 2, (5,)))
print(real_variance)

0.03622810783400194


In [57]:
def clt(n):
    experiments = np.vectorize(lambda _: light_weight_reserve(intensities))(np.arange(1, n + 1))
    return np.sqrt(n) * (lln(experiments, n) - real_mean) / np.sqrt(real_variance)

In [62]:
count_experiments = np.full((1000, ), 50)
experiments_clt = np.array([clt(count_experiments[i]) for i in tqdm(range(len(count_experiments)))])
experiments_clt.size


 30%|███       | 302/1000 [00:16<00:38, 18.36it/s]


 60%|██████    | 604/1000 [00:34<00:21, 18.70it/s]


 91%|█████████ | 906/1000 [00:50<00:04, 18.88it/s]


100%|██████████| 1000/1000 [00:55<00:00, 17.95it/s]


1000

In [63]:
px.histogram(experiments_clt)

In [64]:
import scipy.stats as sps

norm_experiments = sps.norm.rvs(size=1000)
px.histogram(norm_experiments, )


In [67]:
params_of_experiment = np.mgrid[10:110:10, 1:1001][0]
params_of_experiment

array([[ 10,  10,  10, ...,  10,  10,  10],
       [ 20,  20,  20, ...,  20,  20,  20],
       [ 30,  30,  30, ...,  30,  30,  30],
       ...,
       [ 80,  80,  80, ...,  80,  80,  80],
       [ 90,  90,  90, ...,  90,  90,  90],
       [100, 100, 100, ..., 100, 100, 100]])

In [71]:
def get_clt_experiment(count_experiments):
    return np.array([clt(count_experiments[i]) for i in range(len(count_experiments))])

In [76]:
clt_experiments = [get_clt_experiment(param) for param in tqdm(params_of_experiment)]

100%|██████████| 10/10 [08:44<00:00, 52.40s/it]


In [79]:

clt_experiments = np.array(clt_experiments)
clt_experiments_n = clt_experiments[:,:1000]
norm_experiments_n = norm_experiments[:1000]

In [80]:
clt_experiments_n[0].size

1000

In [81]:
norm_experiments_n.size

1000

In [82]:
import plotly.graph_objects as go

def get_histogram(data, left, right, count_bins):
    bins, step = np.linspace(left, right, count_bins + 2, retstep=True)
    bins_points = .5 * (bins[:-1] + bins[1:])
    density, bins = np.histogram(data, bins=bins, density=True)
    density /= density.sum()
    return bins_points, density


$$\text{$A_j$ is a j bin}$$\
$$v_j = \sum_{i=0}^{1000}\mathit{I}(clt_i \in A_j)$$\
$$u_j = \sum_{i=0}^{1000}\mathit{I}(norm_i \in A_j)$$\
$$\text{Distance between distributions}$$\
$$\text{D} = \sum_{j=0}^{39}(v_j - u_j)^2$$

In [83]:
_, norm_density = get_histogram(norm_experiments_n, -4, 4, 39)
np.sum((get_histogram(clt_experiments_n[0], -4, 4, 39)[1] - norm_density)**2) 

0.0010800000000000007

In [84]:
import copy

layout = dict(
    title=r"$\text{Central limit theorem visualization}$",
    barmode="overlay",
    hovermode="closest",
    updatemenus=[dict(
    type="buttons",
    buttons=[dict(label="Play", method="animate",
                  args=[None, 
                        {"frame": {"duration": 1000, "redraw": True},
                         "transition": {"duration": 300, "easing": "quadratic-in-out"}
                        }],
                 ),
             dict(label="Pause", method="animate",
                  args=[None,
                           {"frame": {"duration": 0, "redraw": False},
                            "mode": "immediate",
                            "transition": {"duration": 0}}],
                         )],
    )],
    annotations = [dict(x=2.2, y=0.075,
                        text=repr("$Distance = " + "{:.5f}".format(np.sum(
                            (get_histogram(clt_experiments_n[0], -4, 4, 39)[1] - norm_density)**2
                        )) + "$"),
                        showarrow=False,
                        font=dict(size=14)),
                  ]
)

frames = list()

for i in range(len(clt_experiments)):
    frame_layout = copy.deepcopy(layout)
    layout["annotations"] = [dict(x=2.2, y=0.075,
                                  text=repr("$Distance = " + "{:.5f}".format(np.sum(
                                      (get_histogram(clt_experiments_n[i], -4, 4, 39)[1] - norm_density)**2
                                  )) + "$"),
                                  showarrow=False,
                                  font=dict(size=14)),
                            ]
    frames.append(go.Frame(
        data = [
        go.Histogram(name=r"$\text{Standard normal distribution}$", 
                     x=norm_experiments_n, xbins=go.histogram.XBins(size=.2), 
                     histnorm="probability"),
        go.Histogram(name=repr("$\text{CLT with } n = "+ str((i + 1) * 10) +"$"), 
                     x=clt_experiments_n[i], xbins=go.histogram.XBins(size=.2), 
                     histnorm="probability"),
        ],
        layout=frame_layout
    ))
    

fig = go.Figure(
    data = [
        go.Histogram(name=r"$\text{Standard normal distribution}$", x=norm_experiments_n, xbins=go.histogram.XBins(size=.2), histnorm="probability"),
        go.Histogram(name=repr("$\text{CLT with } n = 10$"), x=clt_experiments_n[0], xbins=go.histogram.XBins(size=.2), histnorm="probability")
    ],
    layout=go.Layout(layout),
    frames=frames,
)
fig.update_traces(opacity=0.75)
fig.show()

$$\large{Task^2}$$\
$$\large{\lambda = (1, 2, 2, 4)}$$\
$$\large{\Lambda = (5, 5, 7, 11)}$$\
$$\large{main = ?}$$

In [85]:
import copy


In [86]:
from itertools import permutations


intensities = [
    {"warm_up_intensity": 1, "main_intensity": 5, "is_main": False},
    {"warm_up_intensity": 2, "main_intensity": 5, "is_main": False},
    {"warm_up_intensity": 2, "main_intensity": 7, "is_main": False},
    {"warm_up_intensity": 4, "main_intensity": 11, "is_main": False},
]

def initialize_system(system):
    system = copy.deepcopy(system)
    system[0]["is_main"] = True
    return system

systems = np.array([initialize_system(system) for system in list(permutations(intensities, 4))])
systems

array([[{'warm_up_intensity': 1, 'main_intensity': 5, 'is_main': True},
        {'warm_up_intensity': 2, 'main_intensity': 5, 'is_main': False},
        {'warm_up_intensity': 2, 'main_intensity': 7, 'is_main': False},
        {'warm_up_intensity': 4, 'main_intensity': 11, 'is_main': False}],
       [{'warm_up_intensity': 1, 'main_intensity': 5, 'is_main': True},
        {'warm_up_intensity': 2, 'main_intensity': 5, 'is_main': False},
        {'warm_up_intensity': 4, 'main_intensity': 11, 'is_main': False},
        {'warm_up_intensity': 2, 'main_intensity': 7, 'is_main': False}],
       [{'warm_up_intensity': 1, 'main_intensity': 5, 'is_main': True},
        {'warm_up_intensity': 2, 'main_intensity': 7, 'is_main': False},
        {'warm_up_intensity': 2, 'main_intensity': 5, 'is_main': False},
        {'warm_up_intensity': 4, 'main_intensity': 11, 'is_main': False}],
       [{'warm_up_intensity': 1, 'main_intensity': 5, 'is_main': True},
        {'warm_up_intensity': 2, 'main_intensity'

In [87]:
def get_experiments_system(system, n):
    return np.vectorize(lambda _: light_weight_reserve(system))(np.empty(n))

def get_experiments_systems(systems, n):
    return np.array([get_experiments_system(system, n) for system in systems])

experiments_systems = get_experiments_systems(systems, 1000)
experiments_systems

array([[0.65505323, 0.38427909, 0.36280261, ..., 0.63428686, 0.83202332,
        0.41991522],
       [0.55600541, 0.12544621, 0.18434358, ..., 0.45803558, 0.62651868,
        0.16994803],
       [0.42534907, 0.26175753, 0.81231653, ..., 0.39837245, 0.35783783,
        0.33119593],
       ...,
       [0.23793637, 0.21686952, 0.2907841 , ..., 0.33803407, 0.29132834,
        0.39916469],
       [0.7399896 , 0.35128383, 0.17006248, ..., 1.31850565, 0.54319731,
        0.3842229 ],
       [0.1909142 , 0.28709571, 0.33934145, ..., 0.43831721, 0.10737876,
        0.46551629]])

In [88]:
def get_means_of_experiments(experiments, axis=1):
    return np.mean(experiments, axis=axis)


sample_means = get_means_of_experiments(experiments_systems)
descriptions_systems = [repr("$"+ ','.join(["(" + str(device["warm_up_intensity"]) + ',' + str(device["main_intensity"]) + ")" for device in system]) + "$") 
                        for system in systems]

px.scatter(x=descriptions_systems, y=sample_means, labels={'x': r'$\text{systems}$', 'y': r'$\text{sample means}$'})

In [90]:
theoretical_averages = np.array([get_means_of_experiments(get_experiments_systems(systems, 100)) for _ in tqdm(range(100))])

100%|██████████| 100/100 [02:58<00:00,  1.78s/it]


In [91]:
averages = theoretical_averages


In [92]:
theoretical_means = np.mean(theoretical_averages.T, axis=1) 

In [93]:
px.scatter(x=descriptions_systems, y=theoretical_means, labels={'x': r'$\text{systems}$', 'y': r'$\text{theoretical means}$'})

In [94]:
px.ecdf(x=experiments_systems[21])